In [40]:
from search import *  
from utils import *
import copy

class State:
    def __init__(self, row, col, roomMatrix):
        self.cleanerRow = row
        self.cleanerCol = col
        self.roomMatrix = roomMatrix
        

class VacuumProblem(Problem):

    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal


    def actions(self, state):
        possible_actions = ['up','left','down','right', 'suck']

        righe = len(state.roomMatrix)
        colonne = len(state.roomMatrix[0])

        if state.cleanerRow in range(1, righe) and state.cleanerCol in range(1, colonne):
            
            if state.roomMatrix[state.cleanerRow - 1][state.cleanerCol] == 'x':
                possible_actions.remove('down')
                
            if state.roomMatrix[state.cleanerRow][state.cleanerCol + 1] == 'x':
                possible_actions.remove('right')
                
            if state.roomMatrix[state.cleanerRow][state.cleanerCol - 1] == 'x':
                possible_actions.remove('left')
         
        else:    
            if state.cleanerRow == righe - 1:
                possible_actions.remove('down')

            if state.cleanerRow == 0:
                possible_actions.remove('up')

            if state.cleanerCol == colonne - 1:
                possible_actions.remove('right')

            if state.cleanerCol == 0:
                possible_actions.remove('left')

        return possible_actions


    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        if action == 'up':
            newRow = state.cleanerRow - 1
            newState = State(newRow, state.cleanerCol, state.roomMatrix) 
        
        elif action == 'down':
            newRow = state.cleanerRow + 1
            newState = State(newRow, state.cleanerCol, state.roomMatrix) 
        
        elif action == 'left':
            newCol = state.cleanerCol - 1
            newState = State(state.cleanerRow, newCol, state.roomMatrix) 
        
        elif action == 'right':
            newCol = state.cleanerCol + 1
            newState = State(state.cleanerRow, newCol, state.roomMatrix) 
        
        elif action == 'suck':
            newMatrix = copy.deepcopy(state.roomMatrix)
            newMatrix[state.cleanerRow][state.cleanerCol] = 'c'
            newState = State(state.cleanerRow, state.cleanerCol, newMatrix)
            
        return newState
       
        
    def goal_test(self, state):
        #ricorda che alla fina l'aspirapolvere deve finire su f
        for roomRow in state.roomMatrix:
            for cell in roomRow:
                if cell == "d" or cell == 'v':
                    return False
        return True
    

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        
        if action == 'suck':
            if state1.roomMatrix[state1.cleanerRow][state1.cleanerRow] == 'v':
                return c + 3
            if state1.roomMatrix[state1.cleanerRow][state1.cleanerRow] == 'd':
                return c + 2
        return c + 1
    

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [41]:
prova = State(0, 0, [['d', 'c'],['c', 'c']])
provaPorbl = VacuumProblem(prova)
newState = provaPorbl.result(provaPorbl.initial,'suck')
print(provaPorbl.goal_test(prova))
print(prova.roomMatrix)
print(newState.roomMatrix)
print(provaPorbl.goal_test(newState))


False
[['d', 'c'], ['c', 'c']]
[['c', 'c'], ['c', 'c']]
True


In [42]:
class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state. Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node. Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1


    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]


    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node


    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]


    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state


    def __hash__(self):
        # We use the hash value of the state
        # stored in the node instead of the node
        # object itself to quickly search a node
        # with the same state in a Hash Table
        return hash(self.state)


    def __repr__(self):
        return "<Node {}>".format(self.state)


    def __lt__(self, node):
        return f(self) < f(node)

In [43]:
fisrtState = State(0, 0, [['d', 'c'],['c', 'c']])
provaPorbl = VacuumProblem(fisrtState)
firstNode = Node(provaPorbl.initial)

children = firstNode.expand(provaPorbl)
for n in children:
    print(n.action)

down
right
suck


In [44]:
firstState = State(0, 0, [['d', 'c'],['c', 'c']])
provaPorbl = VacuumProblem(fisrtState)
firstNode = Node(provaPorbl.initial)

newState = provaPorbl.result(firstState, 'down')
print(newState.cleanerRow)

1


In [45]:
def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

In [46]:
def f(node):
    return node.depth

fisrtState = State(0, 0, [['c', 'x','d'],['c', 'c','c']])
provaPorbl = VacuumProblem(fisrtState)

goal = best_first_graph_search(provaPorbl,lambda n: n.depth, True)
for s in goal.solution():
    print(s)

ValueError: list.remove(x): x not in list

In [8]:
matrix = [['a', 'b'],['c', 'd']]
elem = matrix[1][1]
print(elem)

d
